# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: José Hurtado

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Jose-Hurtado") \
    .master("spark://69ce0b83527f:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import get_json_object

# 1. Read the Data (10 points):
#Load the employees.csv and departments.csv files into PySpark DataFrames.

# Read employees.csv     employee_id id, employee_info JSON
employees_schema = SparkUtils.generate_schema([("employee_id", "string"), ("employee_info", "string")])
employees_df = spark.read \
                .schema(employees_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/employees.csv")
employees_df.show()


# Read departments.csv department_id, department_name, location
departments_schema = SparkUtils.generate_schema([("department_id", "string"), ("department_name", "string"), ("location", "string")])
departments_df = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/departments.csv")
departments_df.show()





+-----------+--------------------+
|employee_id|       employee_info|
+-----------+--------------------+
|          1|{'name': 'Caitlyn...|
|          2|{'name': 'Rachel'...|
|          3|{'name': 'Carrie'...|
|          4|{'name': 'Renee',...|
|          5|{'name': 'Gabriel...|
|          6|{'name': 'Kristen...|
|          7|{'name': 'Jonatha...|
|          8|{'name': 'Michell...|
|          9|{'name': 'Michell...|
|         10|{'name': 'Lisa', ...|
|         11|{'name': 'Cheryl'...|
|         12|{'name': 'Mikayla...|
|         13|{'name': 'Lisa', ...|
|         14|{'name': 'Sean', ...|
|         15|{'name': 'Monica'...|
|         16|{'name': 'Katelyn...|
|         17|{'name': 'Linda',...|
|         18|{'name': 'Tammy',...|
|         19|{'name': 'William...|
|         20|{'name': 'Jorge',...|
+-----------+--------------------+
only showing top 20 rows

+-------------+--------------------+-------------+
|department_id|     department_name|     location|
+-------------+-----------------

In [21]:
# 2. Extract the employee info from the JSON column (20 points):
# Extract the following the columns: name (string), department_id (integer), salary (double), and hire_date (date) from the employee_info column

from pyspark.sql.functions import get_json_object, col

employees_info = employees_df \
    .withColumn("name", get_json_object(employees_df.employee_info, "$.name")) \
    .withColumn("department_id", get_json_object(employees_df.employee_info, "$.department_id")) \
    .withColumn("salary", get_json_object(employees_df.employee_info, "$.salary")) \
    .withColumn("hire_date", get_json_object(employees_df.employee_info, "$.hire_date"))

# Para el cast
employees_info = employees_info \
    .withColumn("department_id", col("department_id").cast("integer")) \
    .withColumn("salary", col("salary").cast("double")) \
    .withColumn("hire_date", col("hire_date").cast("date"))

# Show
employees_info.printSchema()  
employees_info.show(truncate=False)




root
 |-- employee_id: string (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)

+-----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+
|employee_id|employee_info                                                                              |name     |department_id|salary   |hire_date |
+-----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+
|1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |Caitlyn  |103          |115959.78|2002-06-10|
|2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |Rachel   |104          |100820.16|2009

In [22]:
# 3. Join Operations (10 points):
# Join the employees DataFrame with the departments DataFrame on department_id to enrich the employee data with department details.

enriched_df = employees_info.join(departments_df, employees_info.department_id == departments_df.department_id, "inner") \

enriched_df.show()

+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|employee_id|       employee_info|     name|department_id|   salary| hire_date|department_id|     department_name|     location|
+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|          1|{'name': 'Caitlyn...|  Caitlyn|          103|115959.78|2002-06-10|          103| Sales and Marketing|      Chicago|
|          2|{'name': 'Rachel'...|   Rachel|          104|100820.16|2009-07-01|          104|    Data Engineering|      Zapopan|
|          3|{'name': 'Carrie'...|   Carrie|          105|114421.44|1998-12-10|          105|        Data Science|      Seattle|
|          4|{'name': 'Renee',...|    Renee|          104| 54688.13|1995-03-17|          104|    Data Engineering|      Zapopan|
|          5|{'name': 'Gabriel...|Gabriella|          109|106267.03|1995-02-09|          109|    

In [30]:
# 4. Transformations (Using when()) (10 points):
# Add a new column salary_category to the enriched employee DataFrame:
#      If salary is greater than or equal to 55000, set salary_category to "High".
#           Otherwise, set salary_category to "Low".
from pyspark.sql.functions import when

enriched_df = enriched_df.withColumn('category', when(enriched_df['salary'] > 55000, "High").otherwise("Low"))
enriched_df.show()


+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+--------+
|employee_id|       employee_info|     name|department_id|   salary| hire_date|department_id|     department_name|     location|category|
+-----------+--------------------+---------+-------------+---------+----------+-------------+--------------------+-------------+--------+
|          1|{'name': 'Caitlyn...|  Caitlyn|          103|115959.78|2002-06-10|          103| Sales and Marketing|      Chicago|    High|
|          2|{'name': 'Rachel'...|   Rachel|          104|100820.16|2009-07-01|          104|    Data Engineering|      Zapopan|    High|
|          3|{'name': 'Carrie'...|   Carrie|          105|114421.44|1998-12-10|          105|        Data Science|      Seattle|    High|
|          4|{'name': 'Renee',...|    Renee|          104| 54688.13|1995-03-17|          104|    Data Engineering|      Zapopan|     Low|
|          5|{'name': 'Gabriel...|

In [44]:
# 5. Filter and Group (20 points):

# Create two new data frames: one that filters employees with a “High” salary and another that filters employees with a “Low” salary.
high_df = enriched_df.filter(enriched_df['category'] == "High").select("employee_id", "name", "department_name", "salary")
low_df = enriched_df.filter(enriched_df['category'] == "Low").select("employee_id", "name", "department_name", "salary")

# Calculate the average salary per department for the two newly created data frames, 
#which contain the salaries of employees categorized as “High” and “Low.”  
#Resulting data frame for this transformation should contain only department_name and avg_salary columns
hight_department_df = high_df.groupBy("department_name").avg("salary")
low_department_df = low_df.groupBy("department_name").avg("salary")

# Show
#high_df.show()
#low_df.show()
hight_department_df.show()
low_department_df.show()

+--------------------+------------------+
|     department_name|       avg(salary)|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
|    Customer Service|101585.01600000002|
|               Legal|  99366.3129102167|
|        Data Science|101903.63710344829|
|          Operations|100169.65621722837|
|     Human Resources|104999.43191489363|
+--------------------+------------------+

+--------------------+------------------+
|     department_name|       avg(salary)|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
|    Customer Service|42644.472021276604|
|               Legal| 41160.2661

In [50]:
# 6. Sort (10 points): 
from pyspark.sql.functions import desc, asc

# Find the Top 5 employees with highest salaries from employees categorized as “High”
high_employees = high_df.sort(desc("salary")).limit(5)

# Find the Top 5 employees with highest salaries from employees categorized as “Low”
low_employees = low_df.sort(desc("salary")).limit(5)


high_employees.show()
low_employees.show()

+-----------+---------+--------------------+---------+
|employee_id|     name|     department_name|   salary|
+-----------+---------+--------------------+---------+
|       1778|Gabriella|     Human Resources|149989.73|
|       3621|Katherine|     Human Resources| 149979.3|
|        346|     Ryan|Corporate Strateg...| 149963.1|
|       3807|  Caitlyn|               Legal|149956.54|
|       3050|     Mark|               Legal|149915.56|
+-----------+---------+--------------------+---------+

+-----------+-----+--------------------+--------+
|employee_id| name|     department_name|  salary|
+-----------+-----+--------------------+--------+
|       3472|Linda|Corporate Strateg...|54993.53|
|       2545|Tammy|    Data Engineering|54991.71|
|        382|Aaron|Finance and Accou...|54989.45|
|       2153|Craig|     Human Resources| 54945.2|
|       3024|Aaron|    Customer Service| 54937.3|
+-----------+-----+--------------------+--------+



In [56]:
# 7. Find the number of employees with more years in the company (15 points).
from pyspark.sql.functions import lit, year

# Compute a new column with the years in company for each employee
#enriched_df.select("hire_date").sort(desc("hire_date")) Obtener anio. Latest = 2025-03-12
#enriched_df.select("hire_date").sort(asc("hire_date")) Obtener anio. Oldest = 1998-03-15

computed_df = enriched_df.withColumn("years", lit(2025) - year("hire_date")).select("employee_id", "name", "years")

# Find the list of employees with more years in company and count them.
employees_years = computed_df.sort(desc("years"))

# employees_years
employees_years.show()

+-----------+--------+-----+
|employee_id|    name|years|
+-----------+--------+-----+
|        436|Jonathan|   37|
|       1189|   Aaron|   37|
|        446|   Jorge|   37|
|        182|    Mark|   37|
|        478| William|   37|
|        120|   Megan|   37|
|        517| Tristan|   37|
|        200|    Luke|   37|
|        520|   Tammy|   37|
|        308|   James|   37|
|        590| Raymond|   37|
|        891|Jennifer|   37|
|        917|    Luke|   37|
|         59|     Ana|   37|
|        927| Katelyn|   37|
|        159|   Sarah|   37|
|        976|   Jorge|   37|
|        336|  Brandy|   37|
|       1008|  Monica|   37|
|        657| Tristan|   37|
+-----------+--------+-----+
only showing top 20 rows



In [ ]:
# Stop the SparkContext
sc.stop()